In [1]:
from llama_index.core.llms import ChatMessage, MessageRole

from src.agents.pipelines import AIService, pipeline


@pipeline(
    name="documentation_pipeline",
    description="Crea documentacion en un formato profesional, organizado y comprensible.",
)
def pipeline_documentation(service: AIService, prompt: str, context: str | None = None) -> str:
    print(f"Executing documentation pipeline with prompt: {prompt}")
    # Aquí se puede implementar la lógica para generar documentación
    messages = []
    if context:
        messages.append(
            ChatMessage(
                content=f"Contexto adicional:\n{context}",
                role=MessageRole.SYSTEM,
            )
        )
    messages.append(
        ChatMessage(
            content=f"Genera un documento profesional y comprensible basado en el siguiente texto:\n{prompt}",
            role=MessageRole.USER,
        )
    )
    return service.llm_slow.chat(messages=messages).content or "Error generando el documento."


@pipeline(
    name="summary_pipeline",
    description="Resume y simplifica el texto, reduciendo su longitud,  para expresar la idea general.",
)
def pipeline_summary(service: AIService, prompt: str, context: str | None = None) -> str:
    print(f"Executing summary pipeline with prompt: {prompt}")
    # Aquí se puede implementar la lógica para generar un resumen
    messages = []

    if context:
        messages.append(
            ChatMessage(
                content=f"Contexto adicional:\n{context}",
                role=MessageRole.SYSTEM,
            )
        )
    messages.append(
        ChatMessage(
            content=f"Resume el siguiente texto de manera concisa y clara:\n{prompt}",
            role=MessageRole.USER,
        )
    )
    return service.llm_slow.chat(messages=messages).content or "Error generando el resumen."


@pipeline(
    name="explanation_pipeline",
    description="Desarrolla toda la informacion y elabora todo lo posible para explicar algo de forma profunda.",
)
def pipeline_extended_text(service: AIService, prompt: str, context: str | None = None) -> str:
    print(f"Executing extended text pipeline with prompt: {prompt}")
    # Aquí se puede implementar la lógica para generar un texto extendido
    messages = []
    if context:
        messages.append(
            ChatMessage(
                content=f"Contexto adicional:\n{context}",
                role=MessageRole.SYSTEM,
            )
        )
    messages.append(
        ChatMessage(
            content=f"Extiende el siguiente texto para hacerlo más completo y detallado:\n{prompt}",
            role=MessageRole.USER,
        )
    )
    return service.llm_slow.chat(messages=messages).content or "Error generando el texto extendido."


@pipeline(
    name="rag_pipeline",
    description="Usando (RAG) y documentos e informacion interna y completa desarrolla y aporta informacion relacionada que pueda ser relevante.",
)
def pipeline_rag(service: AIService, prompt: str, context: str | None = None) -> str:
    print("Executing RAG pipeline with prompt:", prompt)
    # Aquí se puede implementar la lógica para generar una respuesta utilizando RAG

    nodes = service.rag_service.retrieve_data(prompt)
    if context:
        new_prompt = f"Context:\n{context}\nPrompt:\nElabora una respuesta a la siguiente pregunta utilizando los datos recuperados (es mas importante la informacion aportada que cumplir la peticion):\n{prompt}"
    else:
        new_prompt = prompt

    response = service.llm_slow.response_with_data(new_prompt, nodes)
    return response or "Error generando la respuesta RAG."


In [2]:
from src.agents import llm
from src.agents.llm import LLMConfig, LLMService, Provider, RAGConfig, RAGService
from src.llama_ingestion import IngestionPipelineConfig

llm_slow = LLMService.from_config(
    LLMConfig(
        provider=Provider.OPENAI,
        llm_model_name="gpt-4o",
    )
)
llm_fast = LLMService.from_config(
    LLMConfig(
        provider=Provider.OPENAI,
        llm_model_name="gpt-4.1-nano",
    )
)

rag = RAGService.from_config(
    RAGConfig(),
    # No es necesario el IngestionPipeline, solo si no se usa
    # la configuracion de MongoDB o se quieren procesar nuevos documentos
    IngestionPipelineConfig(
        llm=llm_fast.llm,
        documents_path="./Document",
    ),
)
ai_service = AIService(
    llm_slow=llm_slow,
    llm_fast=llm_fast,
    rag_service=rag,
    pipelines=[
        pipeline_documentation(),
        pipeline_summary(),
        pipeline_extended_text(),
        pipeline_rag(),
    ],  # Aquí podrías añadir pipelines si los tienes definidos
)

questions = [
    "quisiera simplificar y resumir del siguiente texto 'La arquitectura tecnológica de SDP es una estructura compleja que integra múltiples componentes y servicios para ofrecer soluciones eficientes y escalables.'",
    "necesito hacer un documento de arquitectura tecnológica para SDP",
    "necesito escribir un correo que en diga 'el proyecto SDP ha salido adelante y sigue todas las previsiones', alargalo y añade descripciones para que sea un texto extenso completo",
    "Necesito un resumen del estado interno de SDP en base haciendo uso de la documentacion e informacion interna",
]


for question in questions:
    print(f"Pregunta: \n{question}")
    pipelines = ai_service.select_pipeline(question)
    print("pipelines seleccionadas:\n", [p.name for p in pipelines])
    # El primer registro es el mas importante,
    # asi que aplicaremos iterativamente los pipelines seleccionados
    # dejando el mas importante para la respuesta final
    response = ""
    for p in pipelines[::-1]:
        response = p.execute(ai_service, question, response)

    print("Response:\n", response)

    print()


100%|██████████| 108/108 [00:34<00:00,  3.15it/s]


Total nodes ingested: 189
Pregunta: 
quisiera simplificar y resumir del siguiente texto 'La arquitectura tecnológica de SDP es una estructura compleja que integra múltiples componentes y servicios para ofrecer soluciones eficientes y escalables.'
pipelines seleccionadas:
 ['summary_pipeline']
Executing summary pipeline with prompt: quisiera simplificar y resumir del siguiente texto 'La arquitectura tecnológica de SDP es una estructura compleja que integra múltiples componentes y servicios para ofrecer soluciones eficientes y escalables.'
Response:
 La arquitectura tecnológica de SDP es una estructura compleja que combina varios componentes y servicios para brindar soluciones eficientes y escalables.

Pregunta: 
necesito hacer un documento de arquitectura tecnológica para SDP
pipelines seleccionadas:
 ['documentation_pipeline']
Executing documentation pipeline with prompt: necesito hacer un documento de arquitectura tecnológica para SDP
Response:
 **Documento de Arquitectura Tecnológica